## source_Extract_Field_class

In [1]:
# -*- coding: utf-8 -*-
from glob import glob
import nltk 
from nltk import tokenize
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import re
from time import time
from unicodedata import normalize
pd.set_option('display.max_colwidth',-1)

In [163]:
import json
import psycopg2
import pg
from datetime import datetime

In [3]:
field = {
      "date_decision": " 21 julio 2017 ", 
      "date_radication_re": "", 
      "expediente_padre": "20178201051872", 
      "num_decision": "5116594", 
      "radication_RE": "", 
      "resolucion": "confirmar", 
      "sspd": "20178200393395", 
      "sspd_fecha": "01112017"
}
databese_info={
             'localhost':'localhost',
             'user':'postgres',
             'pass':'',
             'db':'pqr_electricaribe'
}

In [164]:
class TypeDataBase():
    def __init__(self,json_db):
        self.database = json_db
        self.cn = ''
        self.conect_datbase(json_db)
    def conect_datbase(self,json_db):
        try:
            self.cn = psycopg2.connect("dbname= "
                                  +json_db['db']+
                                  " user= "
                                  +json_db['user']+
                                  " host= "
                                  +json_db['localhost']+
                                  " password= "
                                  +json_db['pass']+" ")
        except psycopg2.DatabaseError as de:
            if self.cn:
                self.cn.rollback()
                print('Error %s'%de)
        return self.cn
       
    def serch_id(self,json):
        self.cn = self.conect_datbase(self.database)
        cur = self.cn.cursor()
        try:
            cur.execute("select id from businesscase where serial='"+json['sspd']+"'")
            id_bu = cur.fetchall()
            if id_bu:
                json['id'] = id_bu[0][0]
                curup = self.cn.cursor()
            try:
                month = {'enero':'Jan','febrero':'feb','marzo':'mar','abril':'apr','mayo':'may'
                         ,'junio':'jun','julio':'jul','agosto':'aug','septiembre':'Sep','octubre':'oct',
                         'noviembre':'nov','diciembre':'dec'}
                date = json['date_decision']
                date = tokenize.word_tokenize(date)
                mes = date[1]
                date = date[0]+' '+month[mes]+' '+date[2]
                
                cur.execute("update businesscase set description=%s  where serial=%s"
                            ,((datetime.strptime(temp,'%d %b %Y')),json['sspd']))
                self.cn.commit()
                id_bu = curup.fetchall()
                if id_bu:
                    json['id'] = str(id_bu[0][0])
            except psycopg2.DatabaseError as de:
                if self.cn:
                    cur=''    
        except psycopg2.DatabaseError as de:
            if self.cn:
                cur=''
        finally:
            if self.cn:
                self.cn.close()
        return json
    def update_casoentescontrol(self,json):
        self.cn = self.conect_datbase(self.database)
        cur = self.cn.cursor()
        try:
            try:
                cur.execute("UPDATE public.casoentescontrol SET  documentoantecesor=%s,numerodecisionempresaeca=%s, numeroradicadoportico=%s WHERE id=%s",
                (json['expediente_padre'],json['num_decision'],json['radication_RE'],json['id']))
                self.cn.commit()
            except KeyError as k:
                if self.cn:
                    cur=''
        except psycopg2.DatabaseError as de:
            if self.cn:
                cur = ''                
        finally:
            if self.cn:
                self.cn.close()


In [165]:
json_ca = {"date_decision": " 16 junio 2017 ", "date_radication_re": "05 julio 2017", 
      "expediente_padre": "20178201055872","num_decision": "5020051", 
      "radication_RE": "re1110201723181", "resolucion": "confirmar", 
      "sspd": "20178200393205", "sspd_fecha": "01112017"}   
objDataBase = TypeDataBase(databese_info)
id_json = objDataBase.serch_id(json_ca)
objDataBase.update_casoentescontrol(json_ca)

In [166]:
month = {'enero':'Jan','febrero':'feb','marzo':'mar','abril':'apr','mayo':'may'
         ,'junio':'jun','julio':'jul','agosto':'aug','septiembre':'Sep','octubre':'oct',
         'noviembre':'nov','diciembre':'dec'}
date='05 junio 2017' 
date=tokenize.word_tokenize(date)
mes = date[1]
temp = date[0]+' '+month[mes]+' '+date[2]
fecha = datetime.datetime.strptime(temp,'%d %b %Y')

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'

## Class_Extract_Field

In [118]:
class extractField():

    def __init__(self,filelist,json_database):
        self.filelist = glob(filelist+'*.txt')
        self.database = json_database
        
    def organizeDocs(self):
        letters = []
        if self.filelist:
            for file in self.filelist:
                #apertura de todos los documentos que concuerden con la condicion
                with open(file, 'r') as txtfile:
                    letters.append((tokenize.word_tokenize(txtfile.read()), re.match('.*/(.*)-.*', file).group(1)))  
            letters = pd.DataFrame(letters, columns=['Text', 'Name'])
            letters = pd.DataFrame([(np.concatenate(letters[letters.Name == name].Text.values), name) 
                                    for name in letters.Name.unique()], columns=['Text', 'Name'])
            self.filelist = letters
        else:
            print('not found file txt')
        return self.filelist
    
    def cleanDocs(self):
        start = time()
        clean_letters = []
        letters = self.filelist   
        try:
            for _, letter in letters.iterrows():
            #proceso de normalizacion
                words = [normalize('NFKD', word.lower()).encode('ascii', 'ignore').decode('utf-8')
                         for word in letter['Text']]
                # is alphabetic character, not is a stopword, not is a custom word, has at least three letters, 
                # has at least one vowel, has at least one consonant
                words = [word for word in words if word not in stopwords.words('spanish')]
                clean_letters.append(' '.join(words))
            #impresion de tiempo de ejecucion
            print('Cleaning took %.2f seconds' % (time() - start))
            clean_letters = pd.DataFrame(clean_letters)
            self.filelist.insert(1,'CleanText', clean_letters)
        except ValueError and AttributeError:
            print('verify route')
            
        return self.filelist
        
    def extractFields(self):
        nom_ar=0
        list_doc=[]
        letters = self.filelist
        while (nom_ar<=(len(letters)-1)):
            cleanText = re.sub('[^\w\s]', '', letters[letters.Name == letters['Name'][nom_ar]].CleanText.values[0])
            nom_ar=nom_ar+1

            search_result  = re.findall('\s*resuelve\s*articulo\s*(primero|1)\s*(\w*)', cleanText)
            # number sspd
            val_sspd=set(re.findall('\s*sspd\s(\d{14})\s+|$', cleanText))
            #date sspd
            val_sspd_fe=set(re.findall('sspd\s*\d*\sfecha\s*(\d*\S*|\d)', cleanText))
            #expedient
            val_expedient=set(re.findall('\s(\d*\e)\s', cleanText))  
            #expedient father
            val_expedient_father=set(re.findall('radicado\spadre\s(\d{14})\s+|$', cleanText))
            #number solve of decision 
            val_solve_decision=re.findall('\s*resuelve\s*articulo\s*(primero|1)\s*\w*\s*decision\s*(administrativa| )\s*(no|n0| )(\d*)\s+|$', cleanText)
            #date decision
            val_decision_fe= re.findall('\s*resuelve\s*articulo\s*(primero|1)\s*\w*\s*decision\s*(administrativa| )\s*(no|n0| )\d*(\s*\d*\s\w*\s\d*\s\d*)+|$', cleanText)
            # number_RE
            val_number_re=set(re.findall('(no|n0| )\s*(re\d*)\s+|$', cleanText)) 
            # RE_date
            val_re_fe=re.findall('(no|n0| )\s*re\d*\s(\d{2}\s\w*\s\d{4})+|$',cleanText)

            dc = {'resolucion':search_result[0][1],
                          'sspd' : list(val_sspd)[1],
                          'sspd_fecha' : list(val_sspd_fe)[0],
                          'expediente_padre' : list(val_expedient_father)[1], 
                          'num_decision':list(val_solve_decision)[0][3],
                          'date_decision' :list(val_decision_fe)[0][3],
                          'radication_RE':list(val_number_re)[0][1],
                          'date_radication_re':list(val_re_fe)[0][1]}
             #save registry one to one
            list_doc.append(dc)
            #objDataBase = TypeDataBase(self.database)
            #id_json = objDataBase.serch_id(dc)
            #objDataBase.update_casoentescontrol(id_json)
    #add the list_doc to attribute of class     
        #letters.insert(1, 'dictionaryDoc', list_doc)
       # self.filelist = letters
        return list_doc
    

In [119]:
obj_test = extractField('/home/jhonex/document_e/',databese_info)
obj_test.organizeDocs()
obj_test.cleanDocs()
obj_test.extractFields()

Cleaning took 8.37 seconds


[{'date_decision': ' 16 junio 2017 ',
  'date_radication_re': '05 julio 2017',
  'expediente_padre': '20178201055872',
  'num_decision': '5020051',
  'radication_RE': 're1110201723181',
  'resolucion': 'confirmar',
  'sspd': '20178200393205',
  'sspd_fecha': '01112017'},
 {'date_decision': ' re1110201722568 11 ',
  'date_radication_re': '11 julio 2017',
  'expediente_padre': '20178201055352',
  'num_decision': '',
  'radication_RE': '',
  'resolucion': 'confirmar',
  'sspd': '20178200393245',
  'sspd_fecha': '01112017'},
 {'date_decision': ' 21 julio 2017 ',
  'date_radication_re': '',
  'expediente_padre': '20178201051872',
  'num_decision': '5116594',
  'radication_RE': '',
  'resolucion': 'confirmar',
  'sspd': '20178200393395',
  'sspd_fecha': '01112017'},
 {'date_decision': ' re1120201722590 22 ',
  'date_radication_re': '09 junio 2017',
  'expediente_padre': '20178201055392',
  'num_decision': '',
  'radication_RE': '',
  'resolucion': 'confirmar',
  'sspd': '20178200393225',
  '

In [105]:
di={}

In [130]:
di = 'resuelve articulo primero decision administrativa no 1221 dsad 122 212'

In [131]:
dic = re.findall('\s*resuelve\s*articulo\s*(primero|1)\s*\w*\s*decision\s*(administrativa| )\s*(no|n0| )\d*(\s*\d*\s\w*\s\d*\s\d*)',di)

In [132]:
dic

[('primero', 'administrativa', 'no', ' 1221 dsad 122 212')]